In [1]:
import torch

# GPU 사용 가능 여부 확인 (True가 나와야 함)
print(torch.cuda.is_available())

# 현재 잡혀있는 GPU 장치 이름 출력
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

True
NVIDIA TITAN Xp


In [74]:
import torch
import numpy as np
import itertools
from collections import defaultdict

class TSPSolverSOVATorch:
    def __init__(self, dist_matrix, bp_iterations=20, damping=0.7, verbose=True, device='cuda'):
        # Device 설정
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        # 데이터 초기화 및 Tensor 변환
        self.dist_matrix = torch.tensor(dist_matrix, dtype=torch.float32, device=self.device)
        self.num_nodes = self.dist_matrix.shape[0]
        self.N = self.num_nodes - 1
        self.depot = self.N
        self.bp_iterations = bp_iterations
        self.damping = damping
        self.verbose = verbose
        self.INF = 1e8
        self.accumulated_bias = torch.zeros((self.N, self.N), device=self.device)
        
        # 비트마스크 전체 크기 (2^N)
        self.num_states = 1 << self.N
        self.FULL_MASK = self.num_states - 1
        
        # S Matrix 계산 (Max - Dist)
        max_dist = torch.max(self.dist_matrix)
        self.S = max_dist - self.dist_matrix
        self.S.fill_diagonal_(-self.INF)
        
        # Messages initialization (N x N)
        self.tilde_rho = torch.zeros((self.N, self.N), device=self.device)
        self.tilde_eta = torch.zeros((self.N, self.N), device=self.device)
        self.tilde_phi = torch.zeros((self.N, self.N), device=self.device)
        
        # Precompute Masks by Population Count (CPU에서 계산 후 GPU 인덱스로 변환)
        # DP 수행 시 같은 '방문 도시 수(popcount)'를 가진 상태끼리 묶어서 병렬 처리
        self.masks_by_popcount = [[] for _ in range(self.N + 1)]
        for mask in range(self.num_states):
            cnt = bin(mask).count('1')
            if cnt <= self.N:
                self.masks_by_popcount[cnt].append(mask)
        
        # GPU Tensor로 변환 (인덱싱용)
        self.masks_by_popcount_t = [
            torch.tensor(m, dtype=torch.long, device=self.device) 
            for m in self.masks_by_popcount
        ]

    def log(self, msg):
        if self.verbose:
            print(msg)

    def _calc_lambda_sum_bias(self):
        """
        [수정됨] Positive Feedback for Score Maximization
        Score(이득)을 최대화하는 문제이므로, 
        방문 확률(rho)이 높은 곳에 가산점(+)을 줘야 경로가 유지됨.
        """
        N = self.N
        
        # 1. Row Sum
        sum_rho_t = torch.sum(self.tilde_rho, dim=1, keepdim=True)
        
        # 2. Gradient 계산 (부호 수정됨!)
        # - 기존: -rho (Negative Feedback) -> 1등 경로 파괴
        # - 수정: +rho (Positive Feedback) -> 1등 경로 강화
        # - 뒤쪽 항: Row Normalization을 위해 빼줌 (Centering)
        
        current_grad = self.tilde_rho - ((N - 1) / N) * sum_rho_t
        
        # 3. Bias 누적
        step_size = 1  # 학습률
        self.accumulated_bias += step_size * current_grad
        
        return self.accumulated_bias

    def _run_trellis_gpu(self):
        """
        GPU Optimized Trellis (Forward/Backward)
        Dictionary 대신 Dense Tensor [2^N, N] 사용
        """
        N, S, depot = self.N, self.S, self.depot
        bias = self._calc_lambda_sum_bias() # (N, N)
        
        # --- [1] Forward (Alpha) ---
        # alpha[mask, last_node]
        alpha = torch.full((self.num_states, N), -self.INF, device=self.device)
        
        # 초기 상태: 마스크 0은 없으므로, 첫 방문 노드들에 대해 설정
        # 0번 step(depot에서 출발)은 미리 처리
        # depot -> node i
        # mask: 1 << i, score: S[depot, i] + bias[0, i] (bias는 시간 t=0)
        
        # t=0 초기화
        start_bias = bias[0] # (N,)
        start_scores = S[depot, :N] + start_bias # (N,)
        
        # 각 i에 대해 mask = 1<<i 위치에 값 할당
        initial_masks = (1 << torch.arange(N, device=self.device))
        alpha[initial_masks, torch.arange(N, device=self.device)] = start_scores # deprecated indexing fix applied conceptually

        # DP Loop (t = 1 to N-1)
        for t in range(1, N):
            current_bias = bias[t] # (N,)
            prev_masks = self.masks_by_popcount_t[t]     # 현재 방문 수 t인 마스크들
            next_masks_indices = self.masks_by_popcount_t[t+1] # 방문 수 t+1인 마스크들 (검증용 혹은 인덱싱용)
            
            # 현재 유효한 점수 가져오기: (Num_Masks, N)
            curr_scores = alpha[prev_masks, :] 
            
            # Transition: (Num_Masks, N_prev) -> (Num_Masks, N_prev, N_next)
            # prev에서 next로 갈 때의 비용 추가
            # S: (N, N), current_bias: (N,)
            # cost[prev, next] = S[prev, next] + current_bias[next]
            transition_cost = S[:N, :N] + current_bias.view(1, N) # (N, N)
            
            # BroadCasting:
            # curr_scores: (M, N, 1)
            # transition:  (1, N, N)
            # candidates:  (M, N, N) -> (M, N_prev, N_next)
            candidates = curr_scores.unsqueeze(2) + transition_cost.unsqueeze(0)
            
            # 유효하지 않은 경로(이미 방문한 노드) 필터링은 bit logic으로 해야 함
            # M개의 마스크 각각에 대해 next_node 비트가 0이어야 함
            # Tensor 연산으로 마스크 계산: (M, 1) | (1 << (N)) 
            # -> (M, N_next)
            
            mask_col = prev_masks.view(-1, 1) # (M, 1)
            shifts = torch.arange(N, device=self.device).view(1, -1) # (1, N)
            next_bit_check = (mask_col & (1 << shifts)) == 0 # (M, N) : 방문 안했으면 True
            
            # 방문한 곳은 -INF 처리
            # (M, N, N) 에서 next node(3번째 차원) 기준으로 마스킹
            valid_mask = next_bit_check.unsqueeze(1).expand(-1, N, -1) # (M, N_prev, N_next)
            candidates = torch.where(valid_mask, candidates, torch.tensor(-self.INF, device=self.device))
            
            # Flatten candidates to scatter
            # 목적지 Mask 계산: old_mask | (1 << next_node)
            # (M, N_next) 행렬 생성
            new_masks_calc = mask_col | (1 << shifts) # (M, N_next)
            
            # Scatter를 위해 데이터 평탄화
            # 우리는 각 (new_mask, next_node)에 대해 Max 값을 구해야 함
            
            # Source values: candidates (M, N, N)
            src_vals = candidates.reshape(-1)
            
            # Destination indices construction
            # alpha는 (2^N, N) 형태. flat index = mask * N + node
            
            # 각 후보의 new_mask: (M, 1, N) -> expand -> (M, N, N)
            new_masks_expanded = new_masks_calc.unsqueeze(1).expand(-1, N, -1).reshape(-1)
            next_nodes_expanded = shifts.expand(len(prev_masks), N, N).reshape(-1) # 틀림, logic 수정 필요
            
            # 정확한 인덱스 생성:
            # Outer loop: Masks(M), Middle: Prev(N), Inner: Next(N)
            # new_mask는 Prev와 무관하게 (Mask | Next)임.
            m_idx = new_masks_calc.unsqueeze(1).expand(-1, N, -1).reshape(-1) # (M*N*N)
            n_idx = torch.arange(N, device=self.device).view(1, 1, N).expand(len(prev_masks), N, -1).reshape(-1)
            
            flat_indices = m_idx * N + n_idx
            
            # scatter_reduce_ (PyTorch 1.12+) 사용
            # alpha.view(-1) 에 src_vals를 max로 업데이트
            alpha.view(-1).scatter_reduce_(0, flat_indices, src_vals, reduce='amax', include_self=True)

        # --- [2] Backward (Beta) ---
        # Beta 역시 Dense Tensor [2^N, N]
        beta = torch.full((self.num_states, N), -self.INF, device=self.device)
        
        # t=N (마지막) 초기화: Depot으로 돌아가는 비용
        # beta[FULL_MASK, i] = S[i, depot]
        beta[self.FULL_MASK, :] = S[:N, depot]
        
        # Backward Loop (t = N-1 down to 0)
        for t in range(N - 1, -1, -1):
            curr_bias = bias[t] # (N,)
            
            # t+1 시점의 마스크들
            next_masks = self.masks_by_popcount_t[t+1]
            if len(next_masks) == 0: continue
            
            # (M_next, N)
            next_beta_vals = beta[next_masks, :] 
            
            # Xi 계산: beta + bias[node]
            # (M_next, N)
            xi_val = next_beta_vals + curr_bias.view(1, N)
            
            # 이번에는 '이전 노드(prev)'를 찾아야 함
            # next_mask에서 next_node 비트를 끈 것이 prev_mask
            # prev_node -> next_node
            
            # (M_next, N_next) -> next_node가 i일 때의 xi값
            # 우리는 모든 가능한 prev_node j에 대해:
            # val = S[j, i] + xi_val[mask, i]
            # update beta[mask ^ (1<<i), j] with max(val)
            
            # S: (N, N) (row=prev, col=next)
            # xi_val: (M, N) (row=mask, col=next)
            
            # Broadcasting 합: (N, N) + (M, 1, N) -> (M, prev, next) ?
            # (M, 1, N_next) + (1, N_prev, N_next) -> (M, N_prev, N_next)
            
            candidates = xi_val.unsqueeze(1) + S[:N, :N].unsqueeze(0)
            
            # Valid Check: prev_node가 next_mask에 포함되어 있어야 함
            # (역추적이니까, next_mask에 있는 비트들 중 하나가 prev_node가 됨... 아님)
            # 정확히는: next_mask에서 i를 뺀 것이 t시점의 mask.
            # 그리고 그 t시점 mask에 j가 포함되어 있어야 j -> i가 가능?
            # 아니, Backward는 "미래에 i를 방문했고 상태가 next_mask였다면, 현재 j에서의 가치"
            # 즉, next_mask 상태는 j를 이미 방문한 상태여야 함.
            
            mask_col = next_masks.view(-1, 1) # (M, 1)
            shifts = torch.arange(N, device=self.device).view(1, -1) # (1, N)
            
            # next_node(i)가 mask에 포함되어 있어야 유효한 출발점
            has_next_node = (mask_col & (1 << shifts)) != 0 # (M, N_next)
            
            # prev_mask 계산: mask ^ (1<<i)
            prev_masks_calc = mask_col ^ (1 << shifts)
            
            # j (prev_node)가 prev_mask에 포함되어 있어야 함 (단, t=0일 땐 prev_mask가 0이므로 예외)
            if t > 0:
                 # (M, N_next, 1) & (1, 1, N_prev)
                 prev_has_j = (prev_masks_calc.unsqueeze(2) & (1 << shifts).unsqueeze(0).unsqueeze(0)) != 0
                 # (M, N_next, N_prev)
            else:
                 # t=0이면 prev_mask는 0이어야 하고, prev_node는 없음(Depot).
                 # 하지만 코드 구조상 t=0까지 루프를 돌며 beta[0, :] 등을 채움?
                 # 원본 코드는 t=0까지 돌고, cands=[depot] 처리함.
                 # 여기선 NxN 구조만 다루므로 t=0 단계는 사실상 beta update 필요 없음 (depot 연결은 별도)
                 pass
            
            # --- Backward Scatter Logic ---
            # candidates: (M, N_prev, N_next) = Cost(j->i) + Beta(next_mask, i)
            # Target: beta[prev_mask, j]
            
            # 유효성 마스킹
            valid_mask = has_next_node.unsqueeze(1).expand(-1, N, -1) # i가 mask에 있어야함
            if t > 0:
                valid_mask = valid_mask & prev_has_j
            
            # t=0일때는 prev_mask가 0. (0, j)에 업데이트? 
            # 사실 t=0에서 beta값은 필요 없거나 depot 연결용. 여기선 생략 가능하지만 구조 유지.
            
            vals = torch.where(valid_mask, candidates, torch.tensor(-self.INF, device=self.device))
            
            # Scatter Indices
            # Target Mask: prev_masks_calc (M, N_next) -> (M, N_prev, N_next) 확장
            p_mask_idx = prev_masks_calc.unsqueeze(1).expand(-1, N, -1).reshape(-1)
            p_node_idx = torch.arange(N, device=self.device).view(1, N, 1).expand(len(next_masks), -1, N).reshape(-1)
            
            flat_indices = p_mask_idx * N + p_node_idx
            src_vals = vals.reshape(-1)
            
            beta.view(-1).scatter_reduce_(0, flat_indices, src_vals, reduce='amax', include_self=True)

        # --- [3] Soft Output (Delta) Calculation ---
        # Vectorized Max-In / Max-Out
        tilde_delta = torch.zeros((N, N), device=self.device)
        
        # 모든 시간 t에 대해 병렬 처리는 메모리 부담, t별 Loop
        for t in range(N):
            # Alpha(t) + Beta(t) = Total Score Map
            # shape: (Masks_t, N)
            curr_masks = self.masks_by_popcount_t[t+1] # t+1 시점의 alpha, beta 사용 (원본 코드 참조)
            if len(curr_masks) == 0: continue
            
            # alpha: (M, N), beta: (M, N)
            a = alpha[curr_masks]
            b = beta[curr_masks]
            
            # 유효한 값들만 더함 (-INF 방지)
            valid = (a > -self.INF/2) & (b > -self.INF/2)
            scores = torch.where(valid, a + b, torch.tensor(-self.INF, device=self.device))
            
            # City Max Scores: 해당 시간 t에 각 도시 i를 방문했을 때의 최대 점수
            # Reduce over Masks dimension -> (N,)
            city_max_scores = torch.max(scores, dim=0)[0] # (N,)
            
            # Find Best and Second Best for Max-Out
            # topk (2 values)
            top2_vals, top2_idxs = torch.topk(city_max_scores, 2)
            global_max = top2_vals[0]
            global_second = top2_vals[1]
            best_idx = top2_idxs[0]
            
            # Rho bias
            rho_t = self.tilde_rho[t]
            lam_i_for_i = -(1.0/N) * rho_t
            lam_i_for_j = ((N-1.0)/N) * rho_t
            
            max_in = city_max_scores - lam_i_for_i
            
            # Max Out Vectorized
            # 기본적으로 global_max 사용, best_idx 위치만 global_second 사용
            max_out_raw = torch.full((N,), global_max, device=self.device)
            max_out_raw[best_idx] = global_second
            
            max_out = max_out_raw - lam_i_for_j
            
            # Diff calculation
            diff = max_in - max_out
            
            # INF handling
            diff = torch.where(max_in < -self.INF/2, torch.tensor(-self.INF, device=self.device), diff)
            diff = torch.where(max_out < -self.INF/2, torch.tensor(self.INF, device=self.device), diff)
            
            tilde_delta[t] = diff
            
        return alpha, tilde_delta

    def _run_bp_gpu(self, tilde_delta):
        """Matrix Operations Fully on GPU"""
        N = self.N
        
        # 1. Omega 계산
        t_omega = self.tilde_phi + tilde_delta
        
        # 2. Eta 계산 (Column-wise Max excluding self)
        vals, idxs = torch.topk(t_omega, 2, dim=0) # (2, N)
        max1 = vals[0]
        max2 = vals[1]
        argmax = idxs[0]
        
        rows = torch.arange(N, device=self.device).view(N, 1).expand(N, N)
        is_max_pos = (rows == argmax)
        
        new_eta = torch.where(is_max_pos, max2, max1)
        new_eta = -new_eta
        
        # 3. Gamma 계산
        t_gamma = new_eta + tilde_delta
        
        # 4. Phi 계산 (Row-wise Max excluding self)
        vals_r, idxs_r = torch.topk(t_gamma, 2, dim=1)
        max1_r = vals_r[:, 0].unsqueeze(1)
        max2_r = vals_r[:, 1].unsqueeze(1)
        argmax_r = idxs_r[:, 0].unsqueeze(1)
        
        cols = torch.arange(N, device=self.device).view(1, N).expand(N, N)
        is_max_pos_r = (cols == argmax_r)
        
        new_phi = torch.where(is_max_pos_r, max2_r, max1_r)
        new_phi = -new_phi
        
        # Update with Damping
        self.tilde_eta = self.damping * self.tilde_eta + (1 - self.damping) * new_eta
        self.tilde_phi = self.damping * self.tilde_phi + (1 - self.damping) * new_phi
        
        # ---------------------------------------------------------
        # [수정됨] Double Centering 적용
        # ---------------------------------------------------------
        # 1. Rho 계산
        raw_rho = self.tilde_eta + self.tilde_phi
        
        # 2. Row Centering (각 시간 t에 대해 평균 0)
        # 식 (4.2) 의 조건과 유사하게 Row sum constraints 보정
        row_mean = torch.mean(raw_rho, dim=1, keepdim=True)
        raw_rho -= row_mean
        
        # 3. Column Centering (각 도시 i에 대해 평균 0) - 이게 핵심!
        # 특정 도시로 메시지가 쏠리는 것을 방지
        col_mean = torch.mean(raw_rho, dim=0, keepdim=True)
        raw_rho -= col_mean
        
        # 4. 최종 할당
        self.tilde_rho = raw_rho

    def _extract_path_gpu(self, alpha):
            # CPU로 가져와서 순차적으로 역추적 (이 부분은 계산량이 적으므로 CPU가 편함)
            # alpha_cpu = alpha.cpu().numpy() # (주석 처리됨)
            
            path = []
            curr_mask = self.FULL_MASK
            
            # 1) 마지막 도시 선택
            # alpha[FULL_MASK, i] + S[i, depot]
            # S에서도 N개의 도시에 해당하는 부분만 가져오도록 슬라이싱 [:self.N] 필수
            final_scores = alpha[self.FULL_MASK, :] + self.S[:self.N, self.depot]
            best_last = torch.argmax(final_scores).item()
            best_score = final_scores[best_last].item()
            
            if best_score < -self.INF/2:
                return [], self.INF
                
            path = [self.depot, best_last]
            curr_node = best_last
            curr_mask = self.FULL_MASK ^ (1 << best_last)
            
            # 2) 역추적
            for t in range(self.N - 1, 0, -1):
                # alpha[t] 개념이 아니라, alpha[curr_mask, prev] + S[prev, curr_node]
                
                # [수정된 부분] self.S[:, curr_node] -> self.S[:self.N, curr_node]
                # S 행렬은 (N+1 x N+1) 크기이므로, alpha(N)와 크기를 맞추기 위해 
                # 앞쪽 N개의 행(도시들)만 슬라이싱해야 합니다.
                scores = alpha[curr_mask, :] + self.S[:self.N, curr_node]
                
                best_prev = torch.argmax(scores).item()
                val = scores[best_prev].item()
                
                if val < -self.INF/2:
                    print(f"Traceback failed at t={t}")
                    return [], self.INF
                    
                path.append(best_prev)
                curr_node = best_prev
                curr_mask ^= (1 << best_prev)
                
            path.append(self.depot)
            path.reverse()
            
            # Cost Recalculation (CPU)
            dist_cpu = self.dist_matrix.cpu().numpy()
            cost = sum(dist_cpu[path[k], path[k+1]] for k in range(len(path)-1))
            
            return path, cost

    def solve(self):
        best_global_path = []
        best_global_cost = self.INF
        
        print(f"Solving TSP (N={self.N}) with PyTorch/CUDA")
        
        for it in range(self.bp_iterations):
            alpha, tilde_delta = self._run_trellis_gpu()
            path, cost = self._extract_path_gpu(alpha)
            
            if cost < best_global_cost:
                best_global_cost = cost
                best_global_path = path
                print(f"[Iter {it}] Cost: {cost:.2f} (New Best!)")
            elif self.verbose:
                print(f"[Iter {it}] Cost: {cost:.2f}")

            self._run_bp_gpu(tilde_delta)
            
        return best_global_path, best_global_cost

# --- 사용 예시 ---
if __name__ == "__main__":
    # 임의의 거리 행렬 생성 (5x5)
    # 실제 데이터는 파일에서 로드하거나 더 큰 N 사용
    N_CITIES = 15
    data = np.random.rand(N_CITIES, N_CITIES)*100
    # TSP는 대칭이 아니어도 되지만, 보통 대각선 0
    np.fill_diagonal(data, 0)
    
    solver = TSPSolverSOVATorch(data, bp_iterations=20, damping=0.5, verbose=True)
    path, cost = solver.solve()
    print("Final Path:", path)
    print("Final Cost:", cost)

Solving TSP (N=14) with PyTorch/CUDA
[Iter 0] Cost: 207.06 (New Best!)
[Iter 1] Cost: 207.06
[Iter 2] Cost: 207.06
[Iter 3] Cost: 207.06
[Iter 4] Cost: 207.06
[Iter 5] Cost: 207.06
[Iter 6] Cost: 207.06
[Iter 7] Cost: 207.06
[Iter 8] Cost: 207.06
[Iter 9] Cost: 207.06
[Iter 10] Cost: 207.06
[Iter 11] Cost: 207.06
[Iter 12] Cost: 207.06
[Iter 13] Cost: 207.06
[Iter 14] Cost: 207.06
[Iter 15] Cost: 207.06
[Iter 16] Cost: 207.06
[Iter 17] Cost: 207.06
[Iter 18] Cost: 207.06
[Iter 19] Cost: 207.06
Final Path: [14, 12, 7, 9, 13, 10, 1, 0, 6, 3, 5, 2, 4, 8, 11, 14]
Final Cost: 207.05797


In [77]:
import torch
import numpy as np

class TSPSolverSOVATorch:
    def __init__(self, dist_matrix, bp_iterations=20, damping=0.7, verbose=True, device='cuda'):
        # Device 설정
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        # 데이터 초기화 및 Tensor 변환
        self.dist_matrix = torch.tensor(dist_matrix, dtype=torch.float32, device=self.device)
        self.num_nodes = self.dist_matrix.shape[0]
        self.N = self.num_nodes - 1
        self.depot = self.N
        self.bp_iterations = bp_iterations
        self.damping = damping  # BP 메시지 업데이트 시의 관성(Momentum)
        self.verbose = verbose
        self.INF = 1e8
        
        # [변경] accumulated_bias 제거 (Gradient 방식 폐기)
        # 본래 BP 정의대로 매 iter마다 메시지를 새로 계산하여 전달합니다.

        # 비트마스크 전체 크기 (2^N)
        self.num_states = 1 << self.N
        self.FULL_MASK = self.num_states - 1
        
        # S Matrix 계산 (Max - Dist)
        max_dist = torch.max(self.dist_matrix)
        self.S = max_dist - self.dist_matrix
        self.S.fill_diagonal_(-self.INF)
        
        # Messages initialization (N x N)
        self.tilde_rho = torch.zeros((self.N, self.N), device=self.device)
        self.tilde_eta = torch.zeros((self.N, self.N), device=self.device)
        self.tilde_phi = torch.zeros((self.N, self.N), device=self.device)
        
        # Precompute Masks by Population Count
        self.masks_by_popcount = [[] for _ in range(self.N + 1)]
        for mask in range(self.num_states):
            cnt = bin(mask).count('1')
            if cnt <= self.N:
                self.masks_by_popcount[cnt].append(mask)
        
        # GPU Tensor로 변환
        self.masks_by_popcount_t = [
            torch.tensor(m, dtype=torch.long, device=self.device) 
            for m in self.masks_by_popcount
        ]

    def _calc_lambda_original_def(self):
        """
        [Gauge Fixing 제거됨]
        PDF 식 (2.2)의 원래 정의: lambda_{it} = rho_{it}(b_{it})
        
        - 경로가 i를 방문한다면(b=1): rho(1) 메시지를 받음.
        - 따라서 Trellis Score에 rho(1)에 해당하는 값을 '더해줌(+)' (Positive Feedback).
        - 복잡한 Gauge 계수(-(N-1)/N 등)를 제거하고 Belief(tilde_rho)를 그대로 사용.
        """
        # 1. Bias는 Rho (Belief) 그 자체
        # Positive Feedback: 확률이 높은 경로에 가산점을 부여하여 강화
        lambda_val = self.tilde_rho.clone()
        
        # 2. 수치 안정성(Numerical Stability)을 위한 Normalization
        # Gauge Fixing이 아니라, 값이 무한정 커지는 것을 막기 위해 평균만 0으로 맞춤 (BP 표준 기법)
        lambda_val -= torch.mean(lambda_val)
        
        return lambda_val

    def _run_trellis_gpu(self):
        """GPU Optimized Trellis with defined Lambda"""
        N, S, depot = self.N, self.S, self.depot
        
        # [변경] 원래 정의에 따른 Lambda 계산 호출
        bias = self._calc_lambda_original_def() # (N, N)
        
        # --- [1] Forward (Alpha) ---
        alpha = torch.full((self.num_states, N), -self.INF, device=self.device)
        
        # t=0 초기화
        start_bias = bias[0] 
        start_scores = S[depot, :N] + start_bias 
        
        initial_masks = (1 << torch.arange(N, device=self.device))
        alpha[initial_masks, torch.arange(N, device=self.device)] = start_scores

        # DP Loop
        for t in range(1, N):
            current_bias = bias[t] 
            prev_masks = self.masks_by_popcount_t[t]
            
            # (최적화된 Vectorized DP 로직 유지)
            curr_scores = alpha[prev_masks, :] 
            transition_cost = S[:N, :N] + current_bias.view(1, N)
            
            candidates = curr_scores.unsqueeze(2) + transition_cost.unsqueeze(0)
            
            mask_col = prev_masks.view(-1, 1)
            shifts = torch.arange(N, device=self.device).view(1, -1)
            next_bit_check = (mask_col & (1 << shifts)) == 0
            
            valid_mask = next_bit_check.unsqueeze(1).expand(-1, N, -1)
            candidates = torch.where(valid_mask, candidates, torch.tensor(-self.INF, device=self.device))
            
            new_masks_calc = mask_col | (1 << shifts)
            src_vals = candidates.reshape(-1)
            
            m_idx = new_masks_calc.unsqueeze(1).expand(-1, N, -1).reshape(-1)
            n_idx = torch.arange(N, device=self.device).view(1, 1, N).expand(len(prev_masks), N, -1).reshape(-1)
            flat_indices = m_idx * N + n_idx
            
            alpha.view(-1).scatter_reduce_(0, flat_indices, src_vals, reduce='amax', include_self=True)

        # --- [2] Backward (Beta) ---
        beta = torch.full((self.num_states, N), -self.INF, device=self.device)
        beta[self.FULL_MASK, :] = S[:N, depot]
        
        for t in range(N - 1, -1, -1):
            curr_bias = bias[t]
            next_masks = self.masks_by_popcount_t[t+1]
            if len(next_masks) == 0: continue
            
            next_beta_vals = beta[next_masks, :]
            xi_val = next_beta_vals + curr_bias.view(1, N)
            
            candidates = xi_val.unsqueeze(1) + S[:N, :N].unsqueeze(0)
            
            mask_col = next_masks.view(-1, 1)
            shifts = torch.arange(N, device=self.device).view(1, -1)
            has_next_node = (mask_col & (1 << shifts)) != 0
            prev_masks_calc = mask_col ^ (1 << shifts)
            
            if t > 0:
                 prev_has_j = (prev_masks_calc.unsqueeze(2) & (1 << shifts).unsqueeze(0).unsqueeze(0)) != 0
            else:
                 pass
            
            valid_mask = has_next_node.unsqueeze(1).expand(-1, N, -1)
            if t > 0:
                valid_mask = valid_mask & prev_has_j
            
            vals = torch.where(valid_mask, candidates, torch.tensor(-self.INF, device=self.device))
            
            p_mask_idx = prev_masks_calc.unsqueeze(1).expand(-1, N, -1).reshape(-1)
            p_node_idx = torch.arange(N, device=self.device).view(1, N, 1).expand(len(next_masks), -1, N).reshape(-1)
            flat_indices = p_mask_idx * N + p_node_idx
            src_vals = vals.reshape(-1)
            
            beta.view(-1).scatter_reduce_(0, flat_indices, src_vals, reduce='amax', include_self=True)

        # --- [3] Soft Output (Delta) ---
        tilde_delta = torch.zeros((N, N), device=self.device)
        
        for t in range(N):
            curr_masks = self.masks_by_popcount_t[t+1]
            if len(curr_masks) == 0: continue
            
            a = alpha[curr_masks]
            b = beta[curr_masks]
            valid = (a > -self.INF/2) & (b > -self.INF/2)
            scores = torch.where(valid, a + b, torch.tensor(-self.INF, device=self.device))
            
            city_max_scores = torch.max(scores, dim=0)[0]
            
            top2_vals, top2_idxs = torch.topk(city_max_scores, 2)
            global_max = top2_vals[0]
            global_second = top2_vals[1]
            best_idx = top2_idxs[0]
            
            # [변경] Max-In/Max-Out 계산 시에도 Gauge Fix 계수 제거
            # 순수하게 점수 차이(Diff)만 계산
            # 기존 식: lam_i_for_i = -(1.0/N) * rho_t 등 -> 모두 제거
            # Lambda(Bias)는 이미 Trellis 계산에 녹아있으므로, 
            # 여기서는 순수 Trellis Score(city_max_scores)만으로 Delta를 구함
            
            max_in = city_max_scores # Bias 항 제거됨
            
            max_out_raw = torch.full((N,), global_max, device=self.device)
            max_out_raw[best_idx] = global_second
            max_out = max_out_raw # Bias 항 제거됨
            
            diff = max_in - max_out
            
            diff = torch.where(max_in < -self.INF/2, torch.tensor(-self.INF, device=self.device), diff)
            diff = torch.where(max_out < -self.INF/2, torch.tensor(self.INF, device=self.device), diff)
            
            tilde_delta[t] = diff
            
        return alpha, tilde_delta

    def _run_bp_gpu(self, tilde_delta):
        """
        Matrix Operations Fully on GPU
        Constraint Satisfaction을 위한 BP 업데이트 (Doubly Stochastic)
        """
        N = self.N
        
        # 1. Omega
        t_omega = self.tilde_phi + tilde_delta
        
        # 2. Eta (Column Max)
        vals, idxs = torch.topk(t_omega, 2, dim=0)
        max1 = vals[0]
        max2 = vals[1]
        argmax = idxs[0]
        rows = torch.arange(N, device=self.device).view(N, 1).expand(N, N)
        is_max_pos = (rows == argmax)
        new_eta = -torch.where(is_max_pos, max2, max1)
        
        # 3. Gamma
        t_gamma = new_eta + tilde_delta
        
        # 4. Phi (Row Max)
        vals_r, idxs_r = torch.topk(t_gamma, 2, dim=1)
        max1_r = vals_r[:, 0].unsqueeze(1)
        max2_r = vals_r[:, 1].unsqueeze(1)
        argmax_r = idxs_r[:, 0].unsqueeze(1)
        cols = torch.arange(N, device=self.device).view(1, N).expand(N, N)
        is_max_pos_r = (cols == argmax_r)
        new_phi = -torch.where(is_max_pos_r, max2_r, max1_r)
        
        # Update with Damping (Gradient Descent 대신 관성 사용)
        self.tilde_eta = self.damping * self.tilde_eta + (1 - self.damping) * new_eta
        self.tilde_phi = self.damping * self.tilde_phi + (1 - self.damping) * new_phi
        
        # --- Rho Update ---
        raw_rho = self.tilde_eta + self.tilde_phi
        
        # [중요] Gauge Fixing 제거 후 Normalization
        # PDF 식 (4.2)의 Double Centering은 Gauge Fix에서 나온 것이지만,
        # Max-Sum BP에서 값이 무한정 커지는 것을 막고 행/열 제약을 맞추기 위해 
        # 'Sinkhorn' 스타일의 Normalization은 유지하는 것이 수렴에 유리합니다.
        # 하지만 "Gauge Fixing을 걷어내라"는 요청에 따라, 최소한의 수치 안정성(Mean Subtraction)만 남깁니다.
        
        # 1. Row Centering
        raw_rho -= torch.mean(raw_rho, dim=1, keepdim=True)
        # 2. Col Centering
        raw_rho -= torch.mean(raw_rho, dim=0, keepdim=True)
        
        self.tilde_rho = raw_rho

    def _extract_path_gpu(self, alpha):
            path = []
            curr_mask = self.FULL_MASK
            
            # S 행렬 슬라이싱 주의 (N x N)
            final_scores = alpha[self.FULL_MASK, :] + self.S[:self.N, self.depot]
            best_last = torch.argmax(final_scores).item()
            best_score = final_scores[best_last].item()
            
            if best_score < -self.INF/2:
                return [], self.INF
                
            path = [self.depot, best_last]
            curr_node = best_last
            curr_mask = self.FULL_MASK ^ (1 << best_last)
            
            for t in range(self.N - 1, 0, -1):
                scores = alpha[curr_mask, :] + self.S[:self.N, curr_node]
                best_prev = torch.argmax(scores).item()
                val = scores[best_prev].item()
                
                if val < -self.INF/2:
                    return [], self.INF
                    
                path.append(best_prev)
                curr_node = best_prev
                curr_mask ^= (1 << best_prev)
                
            path.append(self.depot)
            path.reverse()
            
            dist_cpu = self.dist_matrix.cpu().numpy()
            cost = sum(dist_cpu[path[k], path[k+1]] for k in range(len(path)-1))
            
            return path, cost

    def solve(self):
        best_global_path = []
        best_global_cost = self.INF
        
        print(f"Solving TSP (N={self.N}) - Original Definition (Lambda=Rho)")
        
        for it in range(self.bp_iterations):
            alpha, tilde_delta = self._run_trellis_gpu()
            path, cost = self._extract_path_gpu(alpha)
            
            if cost < best_global_cost:
                best_global_cost = cost
                best_global_path = path
                print(f"[Iter {it}] Cost: {cost:.2f} (New Best!)")
            elif self.verbose:
                print(f"[Iter {it}] Cost: {cost:.2f}")

            self._run_bp_gpu(tilde_delta)
            
        return best_global_path, best_global_cost

if __name__ == "__main__":
    # N=14 예제 테스트
    N_CITIES = 20
    np.random.seed(42)
    data = np.random.rand(N_CITIES, N_CITIES) * 100
    np.fill_diagonal(data, 0)
    
    solver = TSPSolverSOVATorch(data, bp_iterations=20, damping=0.7, verbose=True)
    path, cost = solver.solve()
    print("Final Path:", path)
    print("Final Cost:", cost)

Solving TSP (N=19) - Original Definition (Lambda=Rho)
[Iter 0] Cost: 142.63 (New Best!)
[Iter 1] Cost: 142.63
[Iter 2] Cost: 142.63
[Iter 3] Cost: 142.63
[Iter 4] Cost: 142.63
[Iter 5] Cost: 142.63
[Iter 6] Cost: 142.63
[Iter 7] Cost: 142.63
[Iter 8] Cost: 142.63
[Iter 9] Cost: 142.63
[Iter 10] Cost: 142.63
[Iter 11] Cost: 142.63
[Iter 12] Cost: 142.63
[Iter 13] Cost: 142.63
[Iter 14] Cost: 142.63
[Iter 15] Cost: 142.63
[Iter 16] Cost: 142.63
[Iter 17] Cost: 142.63
[Iter 18] Cost: 142.63
[Iter 19] Cost: 142.63
Final Path: [19, 7, 5, 9, 1, 12, 4, 18, 14, 10, 8, 11, 17, 13, 2, 16, 15, 0, 6, 3, 19]
Final Cost: 142.62875


In [85]:
import torch
import numpy as np

class TSPSolverSOVATorch:
    def __init__(self, dist_matrix, bp_iterations=20, damping=0.7, verbose=True, device='cuda'):
        # Device 설정
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        # 데이터 초기화 및 Tensor 변환
        self.dist_matrix = torch.tensor(dist_matrix, dtype=torch.float32, device=self.device)
        self.num_nodes = self.dist_matrix.shape[0]
        self.N = self.num_nodes - 1
        self.depot = self.N
        self.bp_iterations = bp_iterations
        self.damping = damping  # BP 메시지 업데이트 시의 관성(Momentum)
        self.verbose = verbose
        self.INF = 1e8
        
        # [변경] accumulated_bias 제거 (Gradient 방식 폐기)
        # 본래 BP 정의대로 매 iter마다 메시지를 새로 계산하여 전달합니다.

        # 비트마스크 전체 크기 (2^N)
        self.num_states = 1 << self.N
        self.FULL_MASK = self.num_states - 1
        
        # S Matrix 계산 (Max - Dist)
        max_dist = torch.max(self.dist_matrix)
        self.S = max_dist - self.dist_matrix
        self.S.fill_diagonal_(-self.INF)
        
        # Messages initialization (N x N)
        self.tilde_rho = torch.zeros((self.N, self.N), device=self.device)
        self.tilde_eta = torch.zeros((self.N, self.N), device=self.device)
        self.tilde_phi = torch.zeros((self.N, self.N), device=self.device)
        
        # Precompute Masks by Population Count
        self.masks_by_popcount = [[] for _ in range(self.N + 1)]
        for mask in range(self.num_states):
            cnt = bin(mask).count('1')
            if cnt <= self.N:
                self.masks_by_popcount[cnt].append(mask)
        
        # GPU Tensor로 변환
        self.masks_by_popcount_t = [
            torch.tensor(m, dtype=torch.long, device=self.device) 
            for m in self.masks_by_popcount
        ]

    def _calc_lambda_original_def(self):
        rho_tilde = self.tilde_rho  # (i, t)
        N = self.N

        # Σ_i ρ̃
        col_sum = rho_tilde.sum(dim=0, keepdim=True)

        # PDF 구조 유지
        bias_per_city = (N - 1)/N * col_sum - rho_tilde  # (a, t)
        bias = bias_per_city.T  # (t, a)

        # (A) 안정성 개선: λ scale down
        c = 0.2
        bias = c * bias

        # (B) 안정성 개선: damping (low-pass filtering)
        if not hasattr(self, "lambda_eff"):
            self.lambda_eff = torch.zeros_like(bias)

        gamma = 0.9
        self.lambda_eff = gamma * self.lambda_eff + (1 - gamma) * bias

        # 발산 방지용 mean subtraction
        self.lambda_eff = self.lambda_eff - self.lambda_eff.mean()

        return self.lambda_eff



    def _run_trellis_gpu(self):
        """GPU Optimized Trellis with defined Lambda"""
        N, S, depot = self.N, self.S, self.depot
        
        # [변경] 원래 정의에 따른 Lambda 계산 호출
        bias = self._calc_lambda_original_def() # (N, N)
        
        # --- [1] Forward (Alpha) ---
        alpha = torch.full((self.num_states, N), -self.INF, device=self.device)
        
        # t=0 초기화
        start_bias = bias[0] 
        start_scores = S[depot, :N] + start_bias 
        
        initial_masks = (1 << torch.arange(N, device=self.device))
        alpha[initial_masks, torch.arange(N, device=self.device)] = start_scores

        # DP Loop
        for t in range(1, N):
            current_bias = bias[t] 
            prev_masks = self.masks_by_popcount_t[t]
            
            # (최적화된 Vectorized DP 로직 유지)
            curr_scores = alpha[prev_masks, :] 
            transition_cost = S[:N, :N] + current_bias.view(1, N)
            
            candidates = curr_scores.unsqueeze(2) + transition_cost.unsqueeze(0)
            
            mask_col = prev_masks.view(-1, 1)
            shifts = torch.arange(N, device=self.device).view(1, -1)
            next_bit_check = (mask_col & (1 << shifts)) == 0
            
            valid_mask = next_bit_check.unsqueeze(1).expand(-1, N, -1)
            candidates = torch.where(valid_mask, candidates, torch.tensor(-self.INF, device=self.device))
            
            new_masks_calc = mask_col | (1 << shifts)
            src_vals = candidates.reshape(-1)
            
            m_idx = new_masks_calc.unsqueeze(1).expand(-1, N, -1).reshape(-1)
            n_idx = torch.arange(N, device=self.device).view(1, 1, N).expand(len(prev_masks), N, -1).reshape(-1)
            flat_indices = m_idx * N + n_idx
            
            alpha.view(-1).scatter_reduce_(0, flat_indices, src_vals, reduce='amax', include_self=True)

        # --- [2] Backward (Beta) ---
        beta = torch.full((self.num_states, N), -self.INF, device=self.device)
        beta[self.FULL_MASK, :] = S[:N, depot]
        
        for t in range(N - 1, -1, -1):
            curr_bias = bias[t]
            next_masks = self.masks_by_popcount_t[t+1]
            if len(next_masks) == 0: continue
            
            next_beta_vals = beta[next_masks, :]
            xi_val = next_beta_vals + curr_bias.view(1, N)
            
            candidates = xi_val.unsqueeze(1) + S[:N, :N].unsqueeze(0)
            
            mask_col = next_masks.view(-1, 1)
            shifts = torch.arange(N, device=self.device).view(1, -1)
            has_next_node = (mask_col & (1 << shifts)) != 0
            prev_masks_calc = mask_col ^ (1 << shifts)
            
            if t > 0:
                 prev_has_j = (prev_masks_calc.unsqueeze(2) & (1 << shifts).unsqueeze(0).unsqueeze(0)) != 0
            else:
                 pass
            
            valid_mask = has_next_node.unsqueeze(1).expand(-1, N, -1)
            if t > 0:
                valid_mask = valid_mask & prev_has_j
            
            vals = torch.where(valid_mask, candidates, torch.tensor(-self.INF, device=self.device))
            
            p_mask_idx = prev_masks_calc.unsqueeze(1).expand(-1, N, -1).reshape(-1)
            p_node_idx = torch.arange(N, device=self.device).view(1, N, 1).expand(len(next_masks), -1, N).reshape(-1)
            flat_indices = p_mask_idx * N + p_node_idx
            src_vals = vals.reshape(-1)
            
            beta.view(-1).scatter_reduce_(0, flat_indices, src_vals, reduce='amax', include_self=True)

        # --- [3] Soft Output (Delta) ---
        tilde_delta = torch.zeros((N, N), device=self.device)
        
        for t in range(N):
            curr_masks = self.masks_by_popcount_t[t+1]
            if len(curr_masks) == 0: continue
            
            a = alpha[curr_masks]
            b = beta[curr_masks]
            valid = (a > -self.INF/2) & (b > -self.INF/2)
            scores = torch.where(valid, a + b, torch.tensor(-self.INF, device=self.device))
            
            city_max_scores = torch.max(scores, dim=0)[0]
            
            top2_vals, top2_idxs = torch.topk(city_max_scores, 2)
            global_max = top2_vals[0]
            global_second = top2_vals[1]
            best_idx = top2_idxs[0]
            
            # [변경] Max-In/Max-Out 계산 시에도 Gauge Fix 계수 제거
            # 순수하게 점수 차이(Diff)만 계산
            # 기존 식: lam_i_for_i = -(1.0/N) * rho_t 등 -> 모두 제거
            # Lambda(Bias)는 이미 Trellis 계산에 녹아있으므로, 
            # 여기서는 순수 Trellis Score(city_max_scores)만으로 Delta를 구함
            
            max_in = city_max_scores # Bias 항 제거됨
            
            max_out_raw = torch.full((N,), global_max, device=self.device)
            max_out_raw[best_idx] = global_second
            max_out = max_out_raw # Bias 항 제거됨
            
            diff = max_in - max_out
            
            diff = torch.where(max_in < -self.INF/2, torch.tensor(-self.INF, device=self.device), diff)
            diff = torch.where(max_out < -self.INF/2, torch.tensor(self.INF, device=self.device), diff)
            
            tilde_delta[t] = diff
            
        return alpha, tilde_delta

    def _run_bp_gpu(self, tilde_delta):
        """
        Matrix Operations Fully on GPU
        Constraint Satisfaction을 위한 BP 업데이트 (Doubly Stochastic)
        """
        N = self.N
        
        # 1. Omega
        t_omega = self.tilde_phi + tilde_delta
        
        # 2. Eta (Column Max)
        vals, idxs = torch.topk(t_omega, 2, dim=0)
        max1 = vals[0]
        max2 = vals[1]
        argmax = idxs[0]
        rows = torch.arange(N, device=self.device).view(N, 1).expand(N, N)
        is_max_pos = (rows == argmax)
        new_eta = -torch.where(is_max_pos, max2, max1)
        
        # 3. Gamma
        t_gamma = new_eta + tilde_delta
        
        # 4. Phi (Row Max)
        vals_r, idxs_r = torch.topk(t_gamma, 2, dim=1)
        max1_r = vals_r[:, 0].unsqueeze(1)
        max2_r = vals_r[:, 1].unsqueeze(1)
        argmax_r = idxs_r[:, 0].unsqueeze(1)
        cols = torch.arange(N, device=self.device).view(1, N).expand(N, N)
        is_max_pos_r = (cols == argmax_r)
        new_phi = -torch.where(is_max_pos_r, max2_r, max1_r)
        
        # Update with Damping (Gradient Descent 대신 관성 사용)
        self.tilde_eta = self.damping * self.tilde_eta + (1 - self.damping) * new_eta
        self.tilde_phi = self.damping * self.tilde_phi + (1 - self.damping) * new_phi
        
        # --- Rho Update ---
        raw_rho = self.tilde_eta + self.tilde_phi
        
        # [중요] Gauge Fixing 제거 후 Normalization
        # PDF 식 (4.2)의 Double Centering은 Gauge Fix에서 나온 것이지만,
        # Max-Sum BP에서 값이 무한정 커지는 것을 막고 행/열 제약을 맞추기 위해 
        # 'Sinkhorn' 스타일의 Normalization은 유지하는 것이 수렴에 유리합니다.
        # 하지만 "Gauge Fixing을 걷어내라"는 요청에 따라, 최소한의 수치 안정성(Mean Subtraction)만 남깁니다.
        
        # 1. Row Centering
        raw_rho -= torch.mean(raw_rho, dim=1, keepdim=True)
        # 2. Col Centering
        raw_rho -= torch.mean(raw_rho, dim=0, keepdim=True)
        
        self.tilde_rho = raw_rho

    def _extract_path_gpu(self, alpha):
            path = []
            curr_mask = self.FULL_MASK
            
            # S 행렬 슬라이싱 주의 (N x N)
            final_scores = alpha[self.FULL_MASK, :] + self.S[:self.N, self.depot]
            best_last = torch.argmax(final_scores).item()
            best_score = final_scores[best_last].item()
            
            if best_score < -self.INF/2:
                return [], self.INF
                
            path = [self.depot, best_last]
            curr_node = best_last
            curr_mask = self.FULL_MASK ^ (1 << best_last)
            
            for t in range(self.N - 1, 0, -1):
                scores = alpha[curr_mask, :] + self.S[:self.N, curr_node]
                best_prev = torch.argmax(scores).item()
                val = scores[best_prev].item()
                
                if val < -self.INF/2:
                    return [], self.INF
                    
                path.append(best_prev)
                curr_node = best_prev
                curr_mask ^= (1 << best_prev)
                
            path.append(self.depot)
            path.reverse()
            
            dist_cpu = self.dist_matrix.cpu().numpy()
            cost = sum(dist_cpu[path[k], path[k+1]] for k in range(len(path)-1))
            
            return path, cost

    def solve(self):
        best_global_path = []
        best_global_cost = self.INF
        
        print(f"Solving TSP (N={self.N}) - Original Definition (Lambda=Rho)")
        
        for it in range(self.bp_iterations):
            alpha, tilde_delta = self._run_trellis_gpu()
            path, cost = self._extract_path_gpu(alpha)
            
            if cost < best_global_cost:
                best_global_cost = cost
                best_global_path = path
                print(f"[Iter {it}] Cost: {cost:.2f} (New Best!)")
            elif self.verbose:
                print(f"[Iter {it}] Cost: {cost:.2f}")

            self._run_bp_gpu(tilde_delta)
            
        return best_global_path, best_global_cost

if __name__ == "__main__":
    # N=14 예제 테스트
    N_CITIES = 20
    np.random.seed(10)
    data = np.random.rand(N_CITIES, N_CITIES) * 100
    np.fill_diagonal(data, 0)
    
    solver = TSPSolverSOVATorch(data, bp_iterations=20, damping=0.7, verbose=True)
    path, cost = solver.solve()
    print("Final Path:", path)
    print("Final Cost:", cost)

Solving TSP (N=19) - Original Definition (Lambda=Rho)
[Iter 0] Cost: 114.97 (New Best!)
[Iter 1] Cost: 114.97
[Iter 2] Cost: 114.97
[Iter 3] Cost: 114.97
[Iter 4] Cost: 114.97
[Iter 5] Cost: 124.71
[Iter 6] Cost: 124.71
[Iter 7] Cost: 124.71
[Iter 8] Cost: 124.71
[Iter 9] Cost: 124.71
[Iter 10] Cost: 124.71
[Iter 11] Cost: 124.71
[Iter 12] Cost: 124.71
[Iter 13] Cost: 124.71
[Iter 14] Cost: 124.71
[Iter 15] Cost: 124.71
[Iter 16] Cost: 124.71
[Iter 17] Cost: 124.71
[Iter 18] Cost: 124.71
[Iter 19] Cost: 124.71
Final Path: [19, 12, 7, 9, 3, 4, 5, 2, 11, 6, 18, 0, 1, 14, 10, 17, 16, 8, 15, 13, 19]
Final Cost: 114.9748


In [14]:
import numpy as np
from collections import defaultdict

class TSPSolverSOVA:
    def __init__(self, dist_matrix, bp_iterations=20, damping=0.7, verbose=True):
        self.dist_matrix = np.array(dist_matrix)
        self.num_nodes = self.dist_matrix.shape[0]
        self.N = self.num_nodes - 1
        self.depot = self.N
        self.bp_iterations = bp_iterations
        self.damping = damping
        self.verbose = verbose
        self.INF = 1e12
        self.FULL_MASK = (1 << self.N) - 1
        
        max_dist = np.max(self.dist_matrix)
        self.S = max_dist - self.dist_matrix
        np.fill_diagonal(self.S, -self.INF) 
        
        # Messages initialization
        self.tilde_rho = np.zeros((self.N, self.N))
        self.tilde_eta = np.zeros((self.N, self.N))
        self.tilde_phi = np.zeros((self.N, self.N))

    def log(self, msg):
        if self.verbose:
            print(msg)

    def _calc_lambda_sum_bias(self):
        """PDF Eq (1.2) Sum(lambda) 계산 (Beta 역할)"""
        N = self.N
        sum_rho_t = np.sum(self.tilde_rho, axis=1, keepdims=True)
        lambda_sum_bias = -self.tilde_rho + ((N - 1) / N) * sum_rho_t
        return lambda_sum_bias

    def _run_trellis(self):
        N, S, depot = self.N, self.S, self.depot
        bias = self._calc_lambda_sum_bias()
        
        # --- [1] Forward (Alpha) ---
        alpha = [defaultdict(lambda: -self.INF) for _ in range(N + 1)]
        alpha[0][(0, depot)] = 0.0
        
        # 최적화: 루프 내 변수 접근 최소화
        for t in range(N):
            current_bias = bias[t]
            next_alpha = alpha[t+1]
            # items() 복사 오버헤드 방지
            for state, prev_score in alpha[t].items():
                if prev_score < -self.INF / 2: continue
                    
                mask, prev_node = state
                
                # 방문 가능한 노드만 순회 (비트 연산 최적화 가능하지만 가독성 유지)
                # N이 작으므로 range(N) 루프는 빠름
                for i in range(N):
                    if not (mask & (1 << i)):
                        new_mask = mask | (1 << i)
                        val = prev_score + S[prev_node, i] + current_bias[i]
                        
                        if val > next_alpha[(new_mask, i)]:
                            next_alpha[(new_mask, i)] = val

        # --- [2] Backward (Beta) ---
        # Beta는 로직 동일, 코드 생략 없이 최적화만 적용
        beta = [defaultdict(lambda: -self.INF) for _ in range(N + 2)]
        final_mask = self.FULL_MASK
        
        for i in range(N):
            beta[N][(final_mask, i)] = S[i, depot]
            
        for t in range(N - 1, -1, -1):
            curr_bias = bias[t]
            curr_beta = beta[t]
            
            for next_state, next_beta_val in beta[t+1].items():
                if next_beta_val < -self.INF / 2: continue
                next_mask, next_node = next_state
                
                # Xi 값 미리 계산
                xi_val = next_beta_val + curr_bias[next_node]
                
                prev_mask = next_mask & ~(1 << next_node)
                
                # 후보군 추출
                if prev_mask == 0:
                    cands = [depot]
                else:
                    cands = [j for j in range(N) if prev_mask & (1 << j)]
                    
                for prev in cands:
                    val = S[prev, next_node] + xi_val
                    state = (prev_mask, prev)
                    if val > curr_beta[state]:
                        curr_beta[state] = val

        # --- [3] Soft Output (Delta) 최적화 (핵심!) ---
        # 기존: O(N^2 * States) -> 최적화: O(States + N^2)
        tilde_delta = np.zeros((N, N))
        
        for t in range(N):
            # 1. 해당 시간 t의 도시별 최대 점수(City Max Scores)를 한 번의 루프로 수집
            city_max_scores = np.full(N, -self.INF)
            
            # alpha와 beta가 존재하는 상태만 빠르게 스캔
            for state, f_score in alpha[t+1].items():
                if f_score < -self.INF / 2: continue
                if state in beta[t+1]:
                    b_score = beta[t+1][state]
                    if b_score > -self.INF / 2:
                        total = f_score + b_score
                        city = state[1]
                        if total > city_max_scores[city]:
                            city_max_scores[city] = total
            
            # 2. '자기 자신 제외 최대값'을 구하기 위한 전처리
            # 전체 최대값과 두 번째 최대값을 찾음
            sorted_indices = np.argsort(city_max_scores)[::-1] # 내림차순 정렬 인덱스
            best_idx = sorted_indices[0]
            second_best_idx = sorted_indices[1]
            
            global_max = city_max_scores[best_idx]
            global_second = city_max_scores[second_best_idx]

            # 3. Delta 계산 (벡터화)
            # lam_i_for_i = -1/N * rho
            # lam_i_for_j = (N-1)/N * rho
            rho_t = self.tilde_rho[t]
            lam_i_for_i = -(1.0/N) * rho_t
            lam_i_for_j = ((N-1.0)/N) * rho_t
            
            # max_in: 내가 선택된 경우의 최대 점수
            max_in = city_max_scores - lam_i_for_i
            
            # max_out: 내가 선택되지 않았을 때(다른 도시 중) 최대 점수
            # 내가 1등이면 -> 2등 점수 사용, 내가 1등 아니면 -> 1등 점수 사용
            max_out_raw = np.full(N, global_max)
            max_out_raw[best_idx] = global_second # 1등 자리는 2등 점수로 교체
            
            max_out = max_out_raw - lam_i_for_j
            
            # 최종 차이 계산 (Inf 처리 포함)
            diff = np.where(max_in < -self.INF/2, -self.INF,
                            np.where(max_out < -self.INF/2, self.INF, max_in - max_out))
            
            tilde_delta[t] = diff
                
        return alpha, tilde_delta

    def _run_bp(self, tilde_delta):
        """
        Numpy Broadcasting을 이용한 BP 완전 최적화
        O(N^3) -> O(N^2)
        """
        N = self.N
        
        # 1. Omega 계산
        t_omega = self.tilde_phi + tilde_delta
        
        # 2. Eta 계산 (Row 방향 Max Excluding Self)
        # 각 열(Column)에서 특정 행(t)을 제외한 최대값 구하기
        # - 전체 최대값(max1)과 두번째 최대값(max2)을 구해서 처리
        col_max_idx = np.argmax(t_omega, axis=0) # 각 열의 최대값 위치(행 인덱스)
        col_max_val = np.max(t_omega, axis=0)    # 각 열의 최대값
        
        # 두 번째 최대값을 구하기 위해 최대값 위치를 -INF로 잠시 변경
        temp_omega = t_omega.copy()
        for c in range(N):
            temp_omega[col_max_idx[c], c] = -self.INF
        col_second_max = np.max(temp_omega, axis=0)
        
        # new_eta 구성
        # t가 최대값 위치가 아니면 -> 최대값 사용
        # t가 최대값 위치면 -> 두 번째 최대값 사용
        new_eta = np.zeros((N, N))
        for t in range(N):
            # t행이 해당 열(c)의 최대값 위치인지 체크
            is_max_pos = (col_max_idx == t)
            # True면 second_max, False면 max_val
            new_eta[t] = np.where(is_max_pos, col_second_max, col_max_val)
        new_eta = -new_eta # 부호 반전

        # 3. Gamma 계산
        t_gamma = new_eta + tilde_delta
        
        # 4. Phi 계산 (Col 방향 Max Excluding Self)
        # 이번엔 각 행(Row)에서 특정 열(i)을 제외한 최대값
        row_max_idx = np.argmax(t_gamma, axis=1)
        row_max_val = np.max(t_gamma, axis=1)
        
        temp_gamma = t_gamma.copy()
        for r in range(N):
            temp_gamma[r, row_max_idx[r]] = -self.INF
        row_second_max = np.max(temp_gamma, axis=1)
        
        new_phi = np.zeros((N, N))
        for i in range(N):
            is_max_pos = (row_max_idx == i)
            # 전치(Transpose) 주의: new_phi[t, i] 이므로 t 루프 대신 열벡터 연산
            # 여기서는 이중 루프 없이 브로드캐스팅을 위해 전치 사용이 헷갈릴 수 있으므로 
            # 단순하게 열 단위 할당
            new_phi[:, i] = np.where(row_max_idx == i, row_second_max, row_max_val)
            
        new_phi = -new_phi # 부호 반전
        
        # Update with Damping
        self.tilde_eta = self.damping * self.tilde_eta + (1 - self.damping) * new_eta
        self.tilde_phi = self.damping * self.tilde_phi + (1 - self.damping) * new_phi
        self.tilde_rho = self.tilde_eta + self.tilde_phi

    def _extract_path(self, alpha):
        path = []
        curr_mask = self.FULL_MASK
        best_score = -self.INF
        best_last = -1

        # 1) 마지막 도시 선택
        for i in range(self.N):
            state = (curr_mask, i)
            if state in alpha[self.N]:
                score = alpha[self.N][state] + self.S[i, self.depot]
                if score > best_score:
                    best_score = score
                    best_last = i

        if best_last == -1:
            return [], self.INF

        # 초기 상태
        path = [self.depot, best_last]
        curr_node = best_last
        curr_mask = self.FULL_MASK ^ (1 << best_last)

        # 2) 역추적
        for t in range(self.N - 1, 0, -1):

            cands = [j for j in range(self.N) if curr_mask & (1 << j)]  # depot 제거

            best_prev = -1
            best_val = -self.INF

            for prev in cands:
                state = (curr_mask, prev)
                if state in alpha[t]:
                    val = alpha[t][state] + self.S[prev, curr_node]   # bias 제거 (이미 α에 포함됨)

                    if val > best_val:
                        best_val = val
                        best_prev = prev

            if best_prev == -1:
                print(f"Traceback failed at t={t}")
                return [], self.INF

            path.append(best_prev)
            curr_node = best_prev
            curr_mask ^= (1 << best_prev)

        path.append(self.depot)
        path.reverse()

        # cost 계산
        cost = sum(self.dist_matrix[path[k], path[k+1]] for k in range(len(path)-1))
        return path, cost

    def solve(self):
        best_global_path = []
        best_global_cost = self.INF
        
        print(f"Solving TSP (N={self.N}) with Alpha Visualization")
        
        for it in range(self.bp_iterations):
            alpha, tilde_delta = self._run_trellis()
            path, cost = self._extract_path(alpha)
            
            if cost < best_global_cost:
                best_global_cost = cost
                best_global_path = path
                print(f"[Iter {it}] Cost: {cost:.2f} (New Best!) | Path: {path}")
            else:
                if self.verbose:
                     print(f"[Iter {it}] Cost: {cost:.2f} | Path: {path}")

            self._run_bp(tilde_delta)
            
        return best_global_path, best_global_cost
    

# --- 사용 예시 ---
if __name__ == "__main__":
    # 임의의 거리 행렬 생성 (5x5)
    # 실제 데이터는 파일에서 로드하거나 더 큰 N 사용
    N_CITIES = 10
    data = np.random.rand(N_CITIES, N_CITIES)
    # TSP는 대칭이 아니어도 되지만, 보통 대각선 0
    np.fill_diagonal(data, 0)
    
    solver = TSPSolverSOVA(data, bp_iterations=20, verbose=True)
    path, cost = solver.solve()
    print("Final Path:", path)
    print("Final Cost:", cost)

Solving TSP (N=9) with Alpha Visualization
[Iter 0] Cost: 1.34 (New Best!) | Path: [9, 3, 0, 5, 2, 1, 7, 4, 8, 6, 9]
[Iter 1] Cost: 1.37 | Path: [9, 5, 2, 1, 0, 3, 6, 8, 4, 7, 9]
[Iter 2] Cost: 1.34 | Path: [9, 3, 0, 5, 2, 1, 7, 4, 8, 6, 9]
[Iter 3] Cost: 1.50 | Path: [9, 1, 7, 4, 8, 6, 2, 3, 0, 5, 9]
[Iter 4] Cost: 1.34 | Path: [9, 5, 0, 3, 6, 8, 4, 2, 1, 7, 9]
[Iter 5] Cost: 1.44 | Path: [9, 3, 6, 8, 7, 4, 2, 1, 0, 5, 9]
[Iter 6] Cost: 1.60 | Path: [9, 4, 2, 1, 0, 3, 6, 8, 7, 5, 9]
[Iter 7] Cost: 1.34 | Path: [9, 3, 0, 5, 2, 1, 7, 4, 8, 6, 9]
[Iter 8] Cost: 1.50 | Path: [9, 1, 7, 4, 8, 6, 2, 3, 0, 5, 9]
[Iter 9] Cost: 1.37 | Path: [9, 5, 2, 1, 0, 3, 6, 8, 4, 7, 9]
[Iter 10] Cost: 1.70 | Path: [9, 4, 8, 7, 5, 2, 1, 0, 3, 6, 9]
[Iter 11] Cost: 1.44 | Path: [9, 3, 6, 8, 7, 4, 2, 1, 0, 5, 9]
[Iter 12] Cost: 1.60 | Path: [9, 4, 2, 1, 0, 3, 6, 8, 7, 5, 9]
[Iter 13] Cost: 1.60 | Path: [9, 4, 2, 1, 0, 3, 6, 8, 7, 5, 9]
[Iter 14] Cost: 1.50 | Path: [9, 1, 7, 4, 8, 6, 2, 3, 0, 5, 9]
[Iter 15] 

In [37]:
import numpy as np
from collections import defaultdict

class TSPSolverSOVA:
    def __init__(self, dist_matrix, bp_iterations=20, damping=0.7, verbose=True):
        self.dist_matrix = np.array(dist_matrix)
        self.num_nodes = self.dist_matrix.shape[0]
        self.N = self.num_nodes - 1
        self.depot = self.N
        self.bp_iterations = bp_iterations
        self.damping = damping
        self.verbose = verbose
        self.INF = 1e12
        self.FULL_MASK = (1 << self.N) - 1
        
        max_dist = np.max(self.dist_matrix)
        self.S = max_dist - self.dist_matrix
        np.fill_diagonal(self.S, -self.INF) 
        
        # Messages initialization
        self.tilde_rho = np.zeros((self.N, self.N))
        self.tilde_eta = np.zeros((self.N, self.N))
        self.tilde_phi = np.zeros((self.N, self.N))

    def log(self, msg):
        if self.verbose:
            print(msg)

    def _calc_lambda_sum_bias(self):
        """PDF Eq (1.2) Sum(lambda) 계산 (Beta 역할)"""
        N = self.N
        sum_rho_t = np.sum(self.tilde_rho, axis=1, keepdims=True)
        lambda_sum_bias = -self.tilde_rho + ((N - 1) / N) * sum_rho_t
        return lambda_sum_bias

    def _run_trellis(self):
        N, S, depot = self.N, self.S, self.depot
        bias = self._calc_lambda_sum_bias()
        
        # --- Forward (Alpha) ---
        alpha = [defaultdict(lambda: -self.INF) for _ in range(N + 1)]
        alpha[0][(0, depot)] = 0.0
        
        for t in range(N):
            current_bias = bias[t]
            for state, prev_score in alpha[t].items():
                if prev_score < -self.INF / 2: continue
                    
                mask, prev_node = state
                for i in range(N):
                    if not (mask & (1 << i)):
                        new_mask = mask | (1 << i)
                        # Alpha Update: 이전 점수 + 거리 점수 + 제약 조건(Bias)
                        val = prev_score + S[prev_node, i] + current_bias[i]
                        
                        if val > alpha[t+1][(new_mask, i)]:
                            alpha[t+1][(new_mask, i)] = val

        # --- Backward (Beta) ---
        beta = [defaultdict(lambda: -self.INF) for _ in range(N + 2)]
        final_mask = (1 << N) - 1
        
        for i in range(N):
            beta[N][(final_mask, i)] = S[i, depot]
            
        for t in range(N - 1, -1, -1):
            for next_state, next_beta_val in beta[t+1].items():
                if next_beta_val < -self.INF / 2: continue
                next_mask, next_node = next_state
                prev_mask = next_mask & ~(1 << next_node)
                
                xi_val = next_beta_val + bias[t][next_node]
                
                cands = [depot] if prev_mask == 0 else [j for j in range(N) if prev_mask & (1<<j)]
                for prev in cands:
                    val = S[prev, next_node] + xi_val
                    if val > beta[t][(prev_mask, prev)]:
                        beta[t][(prev_mask, prev)] = val

        # --- Soft Output (Delta) ---
        tilde_delta = np.zeros((N, N))
        for t in range(N):
            for i in range(N):
                max_in = -self.INF
                max_out = -self.INF
                lam_i_for_i = -(1.0/N) * self.tilde_rho[t, i]
                lam_i_for_j = ((N-1.0)/N) * self.tilde_rho[t, i]

                for state, f_score in alpha[t+1].items():
                    if f_score < -self.INF / 2: continue
                    if state not in beta[t+1]: continue
                    b_score = beta[t+1][state]
                    if b_score < -self.INF / 2: continue
                    
                    total_score = f_score + b_score
                    current_city = state[1]
                    
                    if current_city == i:
                        zeta = total_score - lam_i_for_i
                        if zeta > max_in: max_in = zeta
                    else:
                        zeta = total_score - lam_i_for_j
                        if zeta > max_out: max_out = zeta
                
                if max_in < -self.INF / 2: diff = -self.INF
                elif max_out < -self.INF / 2: diff = self.INF
                else: diff = max_in - max_out
                
                tilde_delta[t, i] = diff
                
        return alpha, tilde_delta

    def _run_bp(self, tilde_delta):
        N = self.N
        t_omega = self.tilde_phi + tilde_delta
        new_eta = np.zeros_like(self.tilde_eta)
        for i in range(N):
            for t in range(N):
                mask = np.ones(N, dtype=bool)
                mask[t] = False
                new_eta[t, i] = -np.max(t_omega[mask, i])
        t_gamma = new_eta + tilde_delta
        new_phi = np.zeros_like(self.tilde_phi)
        for t in range(N):
            for i in range(N):
                mask = np.ones(N, dtype=bool)
                mask[i] = False
                new_phi[t, i] = -np.max(t_gamma[t, mask])
        
        self.tilde_eta = self.damping * self.tilde_eta + (1 - self.damping) * new_eta
        self.tilde_phi = self.damping * self.tilde_phi + (1 - self.damping) * new_phi
        self.tilde_rho = self.tilde_eta + self.tilde_phi

    def _extract_path(self, alpha):
        path = []
        curr_mask = self.FULL_MASK
        best_score = -self.INF
        best_last = -1

        # 1) 마지막 도시 선택
        for i in range(self.N):
            state = (curr_mask, i)
            if state in alpha[self.N]:
                score = alpha[self.N][state] + self.S[i, self.depot]
                if score > best_score:
                    best_score = score
                    best_last = i

        if best_last == -1:
            return [], self.INF

        # 초기 상태
        path = [self.depot, best_last]
        curr_node = best_last
        curr_mask = self.FULL_MASK ^ (1 << best_last)

        # 2) 역추적
        for t in range(self.N - 1, 0, -1):

            cands = [j for j in range(self.N) if curr_mask & (1 << j)]  # depot 제거

            best_prev = -1
            best_val = -self.INF

            for prev in cands:
                state = (curr_mask, prev)
                if state in alpha[t]:
                    val = alpha[t][state] + self.S[prev, curr_node]   # bias 제거 (이미 α에 포함됨)

                    if val > best_val:
                        best_val = val
                        best_prev = prev

            if best_prev == -1:
                print(f"Traceback failed at t={t}")
                return [], self.INF

            path.append(best_prev)
            curr_node = best_prev
            curr_mask ^= (1 << best_prev)

        path.append(self.depot)
        path.reverse()

        # cost 계산
        cost = sum(self.dist_matrix[path[k], path[k+1]] for k in range(len(path)-1))
        return path, cost

    def solve(self):
        best_global_path = []
        best_global_cost = self.INF
        
        print(f"Solving TSP (N={self.N}) with Alpha Visualization")
        
        for it in range(self.bp_iterations):
            alpha, tilde_delta = self._run_trellis()
            path, cost = self._extract_path(alpha)
            
            if cost < best_global_cost:
                best_global_cost = cost
                best_global_path = path
                print(f"[Iter {it}] Cost: {cost:.2f} (New Best!) | Path: {path}")
            else:
                if self.verbose:
                     print(f"[Iter {it}] Cost: {cost:.2f} | Path: {path}")

            self._run_bp(tilde_delta)
            
        return best_global_path, best_global_cost
    
    
# --- 사용 예시 ---
if __name__ == "__main__":
    # 임의의 거리 행렬 생성 (5x5)
    # 실제 데이터는 파일에서 로드하거나 더 큰 N 사용
    N_CITIES = 10
    data = np.random.rand(N_CITIES, N_CITIES)
    # TSP는 대칭이 아니어도 되지만, 보통 대각선 0
    np.fill_diagonal(data, 0)
    
    solver = TSPSolverSOVA(data, bp_iterations=10, damping=0.8, verbose=True)
    path, cost = solver.solve()
    print("Final Path:", path)
    print("Final Cost:", cost)

Solving TSP (N=9) with Alpha Visualization
[Iter 0] Cost: 2.18 (New Best!) | Path: [9, 1, 4, 0, 8, 7, 6, 2, 5, 3, 9]
[Iter 1] Cost: 2.26 | Path: [9, 2, 1, 5, 8, 0, 3, 4, 6, 7, 9]
[Iter 2] Cost: 2.20 | Path: [9, 1, 4, 6, 7, 2, 5, 8, 0, 3, 9]
[Iter 3] Cost: 2.35 | Path: [9, 7, 6, 2, 1, 5, 8, 0, 3, 4, 9]
[Iter 4] Cost: 2.35 | Path: [9, 7, 6, 2, 1, 5, 8, 0, 3, 4, 9]
[Iter 5] Cost: 2.26 | Path: [9, 2, 1, 5, 8, 0, 3, 4, 6, 7, 9]
[Iter 6] Cost: 2.36 | Path: [9, 0, 8, 7, 6, 2, 1, 5, 3, 4, 9]
[Iter 7] Cost: 2.36 | Path: [9, 3, 6, 7, 2, 1, 5, 8, 0, 4, 9]
[Iter 8] Cost: 2.36 | Path: [9, 0, 8, 7, 6, 2, 1, 5, 3, 4, 9]
[Iter 9] Cost: 2.44 | Path: [9, 3, 8, 0, 2, 1, 5, 6, 7, 4, 9]
Final Path: [9, 1, 4, 0, 8, 7, 6, 2, 5, 3, 9]
Final Cost: 2.1835786372086643


In [ ]:
import numpy as np

NEG = -1e12  # numerical -inf


class TSPHypercubeBCJR_SOVA:
    """
    PDF 원래 식 기반으로 다시 정리한 BCJR + SOVA 버전.

    - 단일 trellis ψ[t, mask, last]만 사용 (with-β)
    - α_t(a)는 항상 ψ_{t-1}에서 직접 계산:
        α_t(a) = max_{m_{t-1}, last ∈ m_{t-1}}
                    [ ψ_{t-1}(m_{t-1}, last) + s(last, a) ]
      (여기에는 β_t는 안 들어가고, 과거 시점 ≤ t-1의 β 정보만 포함)
    - trellis → X_{it} 메시지:
        ζ_{it}(a) = α_t(a) + β_t(a) - λ_{it}(a)
      (∑_{i'≠i} λ_{i't}(a) = β_t(a) - λ_{it}(a) 사용한 형태)
    - ψ는 with-β forward trellis:
        ψ_t(m_t, a_t) = max_{m_{t-1}, a_{t-1}}
            [ ψ_{t-1}(m_{t-1}, a_{t-1}) + s(a_{t-1}, a_t) + β_t(a_t) ]
      (코드에서는 t=1..T, β_t(a_t)를 beta[t-1, a_t]에 매핑)
    - BCJR backward bwd[t, mask, last]와 함께
      Γ_t(m_t,last) = ψ_t(m_t,last) + bwd_t(m_t,last)로 SOVA LLR 계산.
    """

    def __init__(self, D, start_city=None,
                 damping=0.3, iters=200, verbose=False,
                 tiny_tiebreak=False, seed=0,
                 patience_no_cost_change=10, cost_tol=1e-12,
                 kappa_bcjr=1.0,
                 damp_L=0.5, damp_beta=0.5, damp_zeta=0.5):

        D = np.array(D, dtype=float)
        assert D.shape[0] == D.shape[1], "D must be square"
        C = D.shape[0]

        if start_city is None:
            start_city = 0
        start_city = int(start_city)
        assert 0 <= start_city < C

        # permute: start -> last (internal depot)
        perm = np.arange(C)
        if start_city != C - 1:
            perm[start_city], perm[C - 1] = perm[C - 1], perm[start_city]
        inv_perm = np.empty(C, dtype=int)
        inv_perm[perm] = np.arange(C)

        self.orig_D = D
        self.D = D[perm][:, perm]
        self.perm = perm
        self.inv_perm = inv_perm

        self.C = C
        self.N = C - 1
        self.depot = C - 1

        self.verbose = verbose
        self.damp = float(damping)
        self.iters = int(iters)
        self.tiny_tiebreak = bool(tiny_tiebreak)
        self.rng = np.random.default_rng(seed)
        self.patience_no_cost_change = int(patience_no_cost_change)
        self.cost_tol = float(cost_tol)

        # BCJR / SOVA 관련 파라미터
        self.kappa_bcjr = float(kappa_bcjr)
        self.damp_L = float(damp_L)
        self.damp_beta = float(damp_beta)
        self.damp_zeta = float(damp_zeta)

        # similarity (bigger is better)
        mx = np.max(self.D)
        self.s = mx - self.D

        # trellis 크기
        self.T = self.N
        self.M = 1 << self.N

        # 단일 forward trellis ψ (with-β)
        self.psi = np.full((self.T + 1, self.M, self.N), NEG)
        self.backptr = np.full((self.T + 1, self.M, self.N, 2), -1, dtype=int)

        # backward trellis (with-β, closure 포함)
        self.bwd_wb = np.full((self.T + 1, self.M, self.N), NEG)

        # α_t(a): ψ_{t-1}로부터 계산
        self.alpha = np.full((self.T, self.N), NEG)

        # simplified messages
        self.gamma_tilde = np.zeros((self.N, self.T))
        self.omega_tilde = np.zeros((self.N, self.T))
        self.phi_tilde   = np.zeros((self.N, self.T))
        self.eta_tilde   = np.zeros((self.N, self.T))
        self.rho_tilde   = np.zeros((self.N, self.T))
        self.delta_tilde = np.zeros((self.N, self.T))

        # λ_t(i,a), ζ_t(i,a), β_t(a)
        self.lambda_ = np.zeros((self.T, self.N, self.N))
        self.zeta    = np.zeros((self.T, self.N, self.N))
        self.beta    = np.zeros((self.T, self.N))

        # damping 캐시
        self._L_prev    = [np.zeros((self.N, self.N)) for _ in range(self.T)]
        self._beta_prev = [np.zeros(self.N)            for _ in range(self.T)]
        self._zeta_prev = [np.zeros((self.N, self.N)) for _ in range(self.T)]

    # ===================== Public =====================
    def run(self):
        stable = 0
        last_cost = None
        best_route, best_cost = None, None

        for it in range(self.iters):
            # (1) forward trellis & α
            self._trellis_forward_and_alpha()

            # (2) backward trellis (primal with β + closure)
            self._trellis_backward_bcjr()

            # (3) BCJR-style SOVA LLR 계산 (T x N)
            llr_t = self._compute_sova_llr_from_bcjr()

            # (4) 메시지 업데이트
            self._update_phi_eta_rho()
            self._update_lambda_beta_zeta_delta(
                kappa=self.kappa_bcjr,
                llr_t=llr_t,
                damp_L=self.damp_L,
                damp_beta=self.damp_beta,
                damp_zeta=self.damp_zeta,
            )
            self._update_gamma_omega()

            if self.tiny_tiebreak:
                self.gamma_tilde += 1e-12 * self.rng.standard_normal(self.gamma_tilde.shape)

            # (5) route & cost
            route = self.estimate_route()
            cost = self._route_cost(route)

            if self.verbose:
                print(f"[{it+1:03d}] cost={cost:.12f} route={route}")

            # best primal 추적
            if best_cost is None or cost < best_cost:
                best_cost, best_route = cost, route

            # plateau early stop
            if last_cost is not None and abs(cost - last_cost) <= self.cost_tol:
                stable += 1
            else:
                stable = 0
            last_cost = cost

            if stable >= self.patience_no_cost_change:
                return best_route, best_cost

        return best_route, best_cost

    # ===================== Trellis (single ψ) & α =====================
    def _trellis_forward_and_alpha(self):
        """
        ψ_t(m_t, last) : with-β forward metric
        α_t(a)         : ψ_{t-1}로부터 계산 (현재 시점 β_t는 포함되지 않음)
        """
        self.psi.fill(NEG)
        self.backptr.fill(-1)
        self.alpha.fill(NEG)

        # t = 1
        # α_1(a) = s(depot, a)
        # ψ_1(m={a}, a) = s(depot, a) + β_1(a)
        for a in range(self.N):
            m = 1 << a
            gain = self.s[self.depot, a]
            self.alpha[0, a] = gain                    # α_1(a)
            self.psi[1, m, a] = gain + self.beta[0, a]  # ψ_1
            self.backptr[1, m, a] = (0, -1)             # 센티넬

        # t = 2..T
        full_mask = (1 << self.N) - 1
        for t in range(2, self.T + 1):
            # 1) α_t(a) 계산: ψ_{t-1}에서 오는 factor→A_t 메시지
            #    α_t(a) = max_{m_{t-1},last∈m_{t-1}, a∉m_{t-1}}
            #                 [ ψ_{t-1}(m_{t-1},last) + s(last,a) ]
            for a in range(self.N):
                best_alpha = NEG
                for mask in range(self.M):
                    if mask == 0 or mask.bit_count() != (t - 1):
                        continue
                    if mask & (1 << a):
                        continue  # 아직 방문 안 한 도시만
                    m = mask
                    while m:
                        last = (m & -m).bit_length() - 1
                        m ^= (1 << last)
                        cand = self.psi[t - 1, mask, last] + self.s[last, a]
                        if cand > best_alpha:
                            best_alpha = cand
                self.alpha[t - 1, a] = best_alpha

            # 2) ψ_t 전이 (with-β)
            for mask in range(self.M):
                if mask == 0 or mask.bit_count() != (t - 1):
                    continue
                for a in range(self.N):
                    if mask & (1 << a):
                        continue
                    new_mask = mask | (1 << a)
                    best = NEG
                    best_last = -1

                    m = mask
                    while m:
                        last = (m & -m).bit_length() - 1
                        m ^= (1 << last)
                        if last == a:
                            continue  # self-loop 금지

                        # ψ_{t-1} + s(last,a) + β_t(a)
                        cand = self.psi[t - 1, mask, last] + self.s[last, a] + self.beta[t - 1, a]
                        if cand > best:
                            best = cand
                            best_last = last

                    if best > self.psi[t, new_mask, a]:
                        self.psi[t, new_mask, a] = best
                        self.backptr[t, new_mask, a] = (mask, best_last)

    # ===================== Backward trellis (BCJR용) =====================
    def _trellis_backward_bcjr(self):
        """
        bwd_wb[t, mask, last]:
          - t 시점에 (mask,last) 상태에서 시작해서
          - 나머지 도시 방문 + depot으로 귀환까지의 최대 future metric.
        전이:
          - t < T:
              bwd[t,mask,last] = max_{a not in mask} { s[last,a] + β_{t+1}(a) + bwd[t+1, mask|{a}, a] }
            (코드에선 β_{t+1}(a)를 beta[t, a]로 저장)
          - t = T:
              full_mask 에 대해서만 closure: s[last, depot]
        """
        self.bwd_wb.fill(NEG)
        full_mask = (1 << self.N) - 1

        # t = T: full mask에서 depot으로 가는 closure
        t = self.T
        for last in range(self.N):
            mask = full_mask
            self.bwd_wb[t, mask, last] = self.s[last, self.depot]

        # t = T-1..1 역순
        for t in range(self.T - 1, 0, -1):
            for mask in range(self.M):
                if mask == 0 or mask.bit_count() != t:
                    continue
                for last in range(self.N):
                    if not (mask & (1 << last)):
                        continue

                    best = NEG
                    avail = (~mask) & full_mask
                    m = avail
                    while m:
                        a = (m & -m).bit_length() - 1
                        m ^= (1 << a)
                        new_mask = mask | (1 << a)
                        cand = self.s[last, a] + self.beta[t, a] + self.bwd_wb[t + 1, new_mask, a]
                        if cand > best:
                            best = cand
                    self.bwd_wb[t, mask, last] = best

    # ===================== SOVA-style LLR from BCJR =====================
    def _compute_sova_llr_from_bcjr(self):
        """
        llr[t, i] ≈
          max_{mask, last=i, |mask|=t+1} [ ψ[t+1, mask, i] + bwd[t+1, mask, i] ]
          - max_{mask, last≠i, |mask|=t+1} [ ψ[t+1, mask, last] + bwd[t+1, mask, last] ]
        여기서 내부 시간 인덱스(t+1)는 1..T 와 매칭, 외부 t는 0..T-1.
        """
        T, N, M = self.T, self.N, self.M
        llr = np.zeros((T, N), dtype=float)

        for t in range(1, T + 1):  # 내부 시간: 1..T
            for i in range(N):
                best_with = NEG
                best_without = NEG

                for mask in range(M):
                    if mask == 0 or mask.bit_count() != t:
                        continue

                    # last = i 인 state
                    val_i = self.psi[t, mask, i] + self.bwd_wb[t, mask, i]
                    if val_i > best_with:
                        best_with = val_i

                    # last ≠ i 인 state들 중 최고값
                    m2 = mask
                    while m2:
                        last = (m2 & -m2).bit_length() - 1
                        m2 ^= (1 << last)
                        if last == i:
                            continue
                        val = self.psi[t, mask, last] + self.bwd_wb[t, mask, last]
                        if val > best_without:
                            best_without = val

                if best_with <= NEG / 2 and best_without <= NEG / 2:
                    llr[t - 1, i] = 0.0
                else:
                    llr[t - 1, i] = best_with - best_without

        return llr

    # ===================== Messages =====================
    def _update_phi_eta_rho(self):
        # φ̃_it = -max_{i'≠i} γ̃_i't
        for t in range(self.T):
            col = self.gamma_tilde[:, t]
            for i in range(self.N):
                self.phi_tilde[i, t] = -np.max(np.delete(col, i)) if self.N > 1 else 0.0
        # η̃_it = -max_{t'≠t} ω̃_it'
        for i in range(self.N):
            row = self.omega_tilde[i, :]
            for t in range(self.T):
                self.eta_tilde[i, t] = -np.max(np.delete(row, t)) if self.T > 1 else 0.0
        # ρ̃_it
        self.rho_tilde = self.eta_tilde + self.phi_tilde

    def _update_lambda_beta_zeta_delta(self, kappa=0.0, llr_t=None,
                                       damp_L=0.5, damp_beta=0.5, damp_zeta=0.5):
        T, N = self.T, self.N

        for t in range(T):
            # (1) rhõ -> (rho0, rho1) 복원
            r = self.rho_tilde[:, t]  # shape (N,)
            rho0 = -r / N             # off-diagonal
            rho1 = rho0 + r           # diagonal

            # (2) L_new(i,a): a==i→rho1, else→rho0
            L_new = np.empty((N, N), float)
            for i in range(N):
                L_new[i, :] = rho0[i]
                L_new[i, i] = rho1[i]

            # (3) λ 이중 센터링
            L_new -= L_new.mean(axis=1, keepdims=True)
            L_new -= L_new.mean(axis=0, keepdims=True)

            # (4) λ damping
            L_prev = self._L_prev[t]
            L = damp_L * L_new + (1 - damp_L) * L_prev
            self.lambda_[t] = L

            # (5) β_t(a) = sum_i λ_{it}(a)
            beta_new = L.sum(axis=0)
            beta_new -= beta_new.mean()

            # (6) β damping
            beta_prev = self._beta_prev[t]
            beta_t = damp_beta * beta_new + (1 - damp_beta) * beta_prev
            self.beta[t, :] = beta_t

            # (7) ζ_it(a) = α_t(a) + β_t(a) - λ_it(a)
            a_t = self.alpha[t, :]
            Z_new = a_t[np.newaxis, :] + beta_t[np.newaxis, :] - L

            # (8) SOVA LLR 주입 (대각 성분)
            if kappa and llr_t is not None:
                for i in range(N):
                    Z_new[i, i] += kappa * llr_t[t, i]

            # (9) ζ damping
            Z_prev = self._zeta_prev[t]
            Z = damp_zeta * Z_new + (1 - damp_zeta) * Z_prev
            self.zeta[t] = Z

            # (10) δ̃_it = ζ_it(i) - max_{a≠i} ζ_it(a)
            for i in range(N):
                zi = Z[i, :]
                self.delta_tilde[i, t] = 0.0 if N == 1 else (zi[i] - np.max(np.delete(zi, i)))

        # 캐시 갱신
        self._L_prev    = [self.lambda_[t].copy() for t in range(T)]
        self._beta_prev = [self.beta[t].copy()    for t in range(T)]
        self._zeta_prev = [self.zeta[t].copy()    for t in range(T)]

    def _update_gamma_omega(self):
        gamma_new = self.eta_tilde + self.delta_tilde
        omega_new = self.phi_tilde + self.delta_tilde
        self.gamma_tilde = self.damp * gamma_new + (1 - self.damp) * self.gamma_tilde
        self.omega_tilde = self.damp * omega_new + (1 - self.damp) * self.omega_tilde

    # ===================== Decode =====================
    def estimate_route(self):
        full_mask = (1 << self.N) - 1
        best_val = NEG
        best_last = -1

        for last in range(self.N):
            base = self.psi[self.T, full_mask, last]
            if base <= NEG / 2:
                continue
            val = base + self.s[last, self.depot]  # closure
            if val > best_val:
                best_val = val
                best_last = last

        # backtrack
        if best_last < 0:
            # fallback: α 기반 greedy
            route_internal = [self.depot]
            used = set()
            for t in range(self.T):
                sc = self.alpha[t].copy()
                for u in used:
                    sc[u] = NEG
                if self.tiny_tiebreak:
                    sc += 1e-15 * np.arange(self.N)
                a = int(np.argmax(sc))
                used.add(a)
                route_internal.append(a)
            route_internal.append(self.depot)
        else:
            route_inner = []
            mask = full_mask
            last = best_last
            t = self.T
            while t > 0 and 0 <= last < self.N:
                route_inner.append(last)
                prev_mask, prev_last = self.backptr[t, mask, last]
                mask, last = prev_mask, prev_last
                t -= 1
            route_inner.reverse()
            route_internal = [self.depot] + route_inner + [self.depot]

        return [int(self.inv_perm[c]) for c in route_internal]

    def _route_cost(self, route):
        return float(sum(self.orig_D[route[k], route[k + 1]] for k in range(len(route) - 1)))


    
# --- 사용 예시 ---     
if __name__ == "__main__":
    # 임의의 거리 행렬 생성 (5x5)
    # 실제 데이터는 파일에서 로드하거나 더 큰 N 사용
    N_CITIES = 10
    data = np.random.rand(N_CITIES, N_CITIES)
    # TSP는 대칭이 아니어도 되지만, 보통 대각선 0
    np.fill_diagonal(data, 0)
    
    solver = TSPHypercubeBCJR_SOVA(data, verbose=True)
    path, cost = solver.run()
    print("Final Path:", path)
    print("Final Cost:", cost)

[001] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[002] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[003] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[004] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[005] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[006] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[007] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[008] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[009] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[010] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
[011] cost=1.343409498996 route=[0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
Final Path: [0, 4, 7, 3, 8, 2, 1, 9, 6, 5, 0]
Final Cost: 1.3434094989958751
